In [0]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())

In [1]:
import tensorflow as tf
# import cv2 as cv
import os
import numpy as np
import glob
import pandas as pd

# # from random import shuffle
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense
from tensorflow.contrib import rnn
from sklearn.utils import shuffle
from tqdm import tqdm
# import tensorflow_hub as hub
# from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [47]:
 drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
tf.__version__

'1.15.0-rc3'

In [0]:
# for filename in glob.glob("/content/drive/My Drive/value labs/DataSet/*.csv"):
  # print(filename)
# df=pd.read_csv("/content/drive/My Drive/value labs/DataSet/Train.csv")  
ques,ans,dist,vocab,label=[],[],[],[],[]
with open("/content/drive/My Drive/valuelabs/DataSet/Train.csv") as fl:
  data=csv.reader(fl)
  for ind,rows in enumerate(data):
    # print(rows)
    if ind!=0:
      a=' '.join(rows).replace("'","").replace("  "," ").replace('"',"").replace(","," ,").strip()
      vocab.append(a)
      ques.append(rows[0].replace("'","").replace("  "," ").replace('"',"").replace(","," ,").strip())
      ans.append(rows[1].replace("'","").replace("  "," ").replace('"',"").replace(","," ,").strip())
      dist.append("SOS "+rows[2].replace("'","").replace("  "," ").replace('"',"").replace(","," ,").strip())
      tp=rows[2].replace("'","").replace("  "," ").replace('"',"").replace(","," ,").strip()
      label.append(tp+" EOS")


In [0]:
vocab.append("EOS")
vocab.append("SOS")

In [6]:
a

'The boy grew up with the smell of the herbs because he often mixed herbs for his mother he liked go play with herbs . , he studied at a herbal medicine store , his family had a herbal medicine store'

In [7]:
vocab[0]

'Meals can be served in rooms at 9:00 p. m. outside the room at 3:00 p. m. , in the dining - room at 6:00 p. m. , in the dining - room from 7:30 a. m. to 9:15 p. m.'

In [8]:
ques[:10]

['Meals can be served',
 'It can be inferred from the passage that',
 'The author called Tommy s parents in order to',
 'It can be inferred from the passage that',
 'How can we deal with snake wounds according to the passage ?',
 'What was the writer s problem when she studied in the boarding school at first ?',
 'Who were killed on February 5 in a small town near Johannesburg ?',
 'According to the writer  , which of the following statements is true ?',
 'During a fire children often',
 'What s the title of the passage ?']

In [9]:
ans[:10]

['in rooms at 9:00 p. m.',
 'The local government can deal with the problem of lacking money by some means .',
 'help them realize their influence on Tommy',
 'the writer is not very willing to use idioms',
 'Stay calm and do nt move .',
 'She missed her family very much .',
 'Chen Jianqing and one of her partners',
 'Soccer is popular all over the world  , but truly great players are few .',
 'panic',
 'Five children died in a kindergarten bus accident .']

In [10]:

dist[:10]

['SOS outside the room at 3:00 p. m. , in the dining - room at 6:00 p. m. , in the dining - room from 7:30 a. m. to 9:15 p. m.',
 'SOS If some tragedies occur again  , relevant departments of the State Council should take responsibility completely . , Currently  , the central government has established sound management systems to guarantee school bus safety . , Central and local governments will share the costs on building more modern schools in rural areas .',
 'SOS blame Tommy for his failing grades , blame Tommy for his failing grades',
 'SOS idioms are the most important part in a language , nonnative speakers should learn more idioms , there are no ways to master idioms',
 'SOS Cut the wound and suck the poison out .',
 'SOS She did nt like her new school . , She did nt get along well with her classmates . , She did nt like her new teacher .',
 'SOS Chen Jianqing and her husband , Chen Jingmin and a Hong Kong businessman , Chen Jingmin and one of his  s',
 'SOS Millions of people 

In [11]:
label[:10]

['outside the room at 3:00 p. m. , in the dining - room at 6:00 p. m. , in the dining - room from 7:30 a. m. to 9:15 p. m. EOS',
 'If some tragedies occur again  , relevant departments of the State Council should take responsibility completely . , Currently  , the central government has established sound management systems to guarantee school bus safety . , Central and local governments will share the costs on building more modern schools in rural areas . EOS',
 'blame Tommy for his failing grades , blame Tommy for his failing grades EOS',
 'idioms are the most important part in a language , nonnative speakers should learn more idioms , there are no ways to master idioms EOS',
 'Cut the wound and suck the poison out . EOS',
 'She did nt like her new school . , She did nt get along well with her classmates . , She did nt like her new teacher . EOS',
 'Chen Jianqing and her husband , Chen Jingmin and a Hong Kong businessman , Chen Jingmin and one of his  s EOS',
 'Millions of people all 

In [0]:
# "," in vocab[0]

In [0]:
# a[0][0]

In [0]:
tok = Tokenizer(num_words=30000,lower=True,filters="#$%()*+-./:;<=>?@[\\]^_`{|}~\t\n")
tok.fit_on_texts(vocab)
seq_ques= tok.texts_to_sequences(ques)
seq_ans=tok.texts_to_sequences(ans)
seq_dist=tok.texts_to_sequences(dist)
seq_label=tok.texts_to_sequences(label)

In [0]:
seq_ques_l=[len(i) for i in seq_ques]
seq_ans_l=[len(i) for i in seq_ans]
seq_dist_l=[len(i) for i in seq_dist]
seq_label_l=[len(i) for i in seq_label]

In [16]:
print(max(seq_ques_l))
print(max(seq_ans_l))
print(max(seq_dist_l))

52
109
129


In [0]:

import statistics as st
stdq=st.stdev(seq_ques_l)
stda=st.stdev(seq_ans_l)
stdd=st.stdev(seq_dist_l)
stdl=st.stdev(seq_label_l)


In [0]:
mq=st.mean(seq_ques_l)
ma=st.mean(seq_ans_l)
md=st.mean(seq_dist_l)
ml=st.mean(seq_label_l)

In [0]:
word2idx = tok.word_index

In [0]:
vocab_len=len(word2idx)+1

In [21]:
word2idx[","]

2

In [0]:
import math
for i,val in enumerate(seq_ques):
  if len(val)> mq+(2*stdq):
    seq_ques[i]=val[:int(mq)+math.ceil((2*stdq))]
  if len(seq_ans[i])> ma+(2*stda):
    seq_ans[i]=seq_ans[i][:int(ma)+math.ceil((2*stda))]
  if len(seq_dist[i])> md+(2*stdd):
    seq_dist[i]=seq_dist[i][:int(md)+math.ceil((2*stdd))]    
    # seq_dist[i][-1]=word2idx["eos"]
  if len(seq_label[i])> ml+(2*stdl):
    seq_label[i]=seq_label[i][:int(ml)+math.ceil((2*stdl))]    
    seq_label[i][-1]=word2idx["eos"]


In [0]:
max_seq_ques=int(mq)+math.ceil((2*stdq))
max_seq_ans=int(ma)+math.ceil((2*stda))
max_seq_dist=int(md)+math.ceil((2*stdd))

In [24]:
print(max_seq_ans)
print(max_seq_ques)
print(max_seq_dist)

14
16
43


In [0]:
data_feed_ques= pad_sequences(seq_ques, maxlen=max_seq_ques+1,padding="post")
data_feed_ans= pad_sequences(seq_ans, maxlen=max_seq_ans+1,padding="post")
data_feed_dist= pad_sequences(seq_dist, maxlen=max_seq_dist+1,padding="post")
data_feed_label= pad_sequences(seq_label, maxlen=max_seq_dist+1,padding="post")


In [26]:
print(data_feed_label.shape)
print(data_feed_ans.shape)
print(data_feed_dist.shape)
print(data_feed_ques.shape)

(31499, 44)
(31499, 15)
(31499, 44)
(31499, 17)


In [0]:
emb={}
with open(os.path.join('/content/drive/My Drive/glove.6B.50d.txt')) as f :
  for l in f:
    v=l.split()
    w=v[0]
    emb[w]=v[1:]

In [0]:
# emb["sos"]

In [29]:
t={}
EMB_DIM=50
num_words = min(40000,len(word2idx)+1) 
embedding_matrix = np.random.rand(num_words,EMB_DIM)*2-1
for word,i in word2idx.items():
        embedding_vector = emb.get(word)
        if embedding_vector is not None:
             embedding_matrix[i] = np.asarray(embedding_vector)
        else:
          t[word]=i
          embedding_matrix[i] = np.asarray(emb.get("unk"))
print(t)
print(len(t))

{'color1': 2128, 'selfies': 3499, 'ks5u': 3992, 'oclock': 4331, 'color1s': 4376, 'alphago': 5135, 'wechat': 5369, 'titanicii': 5931, 'cyberschools': 6040, 'locata': 6330, 'playpump': 6350, 'roosegaarde': 6441, 'nottrue': 6671, 'rmts': 6714, 'trueaccording': 6762, 'douthett': 6790, 'earworms': 6805, 'oneil': 6875, 'greenplug': 6979, 'selfie': 6996, 'tuhao': 7112, 'telp': 7219, 'lampbrella': 7265, 'backster': 7283, 'amusics': 7295, 'weblish': 7385, 'ieoh': 7420, 'youyou': 7486, 'hasbrook': 7555, 'sprayable': 7587, 'tsaatan': 7617, 'color1ful': 7632, 'googlefight': 7699, 'schoo1': 7796, 'lawlis': 7884, 'phubbing': 7988, 'annalisee': 8095, 'siwosz': 8194, 'paragragh': 8246, 'qinmei': 8281, 'tirole': 8290, 'booksfirst': 8311, 'iwfu': 8326, 'dequina': 8412, 'poknemon': 8433, 'cymber': 8447, 'color1ed': 8465, 'rx1e': 8584, 'zxxk': 8586, 'guling': 8648, 'viseskul': 8657, 'kurio': 8661, 'earckson': 8704, 'genzeb': 8721, 'paracutin': 8725, 'johnsy': 8741, 'lovci': 8853, 'politechnic': 8885, 'beo

In [0]:
nn=-6300
data_feed_ans_test=data_feed_ans[-6300:]
data_feed_ans=data_feed_ans[:nn]

data_feed_ques_test=data_feed_ques[-6300:]
data_feed_ques=data_feed_ques[:nn]

data_feed_dist_test=data_feed_dist[-6300:]
data_feed_dist=data_feed_dist[:nn]

data_feed_label_test=data_feed_label[-6300:]
data_feed_label=data_feed_label[:nn]

In [0]:
seq_ques_l_test=seq_ques_l[-6300:]
seq_ques_l=seq_ques_l[:nn]

seq_ans_l_test=seq_ans_l[-6300:]
seq_ans_l=seq_ans_l[:nn]

seq_dist_l_test=seq_dist_l[-6300:]
seq_dist_l=seq_dist_l[:nn]
 
seq_label_l_test=seq_label_l[-6300:]
seq_label_l=seq_label_l[:nn]

In [32]:
data_feed_label.shape

(25199, 44)

In [0]:
def ques_emb(xx,dp,kp):
  
  with tf.variable_scope('ques'):
#     print("kp=",kp[0],kp[1])
    lstms_fw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [400]]
    lstms_bw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [400]]
#     print('fw',lstms_fw)
    drops_fw = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_fw]
    drops_bw= [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_bw]

    lstm_output,output_fw, output_bw =tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=drops_fw,cells_bw=drops_bw,inputs=xx,dtype=tf.float32)
    # print("ques",lstm_output.shape)
    return output_fw, output_bw

In [0]:
def ans_emb(xx,dp,kp):
  
  with tf.variable_scope('ans'):
#     print("kp=",kp[0],kp[1])
    lstms_fw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [400]]
    lstms_bw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [400]]
    print('fw',type(lstms_fw))
    drops_fw = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_fw]
    drops_bw= [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_bw]

    lstm_output,output_fw,output_bw =tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=drops_fw,cells_bw=drops_bw,inputs=xx,dtype=tf.float32)
    # print("ans",lstm_output.shape)
    return output_fw, output_bw

In [0]:
def main_network(fw,bw,xx,dp,kp):
  with tf.variable_scope('final',reuse=tf.AUTO_REUSE):
    lstms_fw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [800]]
    lstms_bw = [tf.nn.rnn_cell.LSTMCell(size ,use_peepholes=True) for size in [800]]
#     print('fw',lstms_fw)
    drops_fw = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_fw]
    drops_bw= [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=kp,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_bw]

    lstm_output,fw,bw =tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=drops_fw,cells_bw=drops_bw,inputs=xx,initial_states_fw=[fw],initial_states_bw=[bw],dtype=tf.float32)
    # print('lstm',np.shape(lstm_output))
    fs=lstm_output
    logits=tf.nn.dropout(fs,keep_prob=dp)
    logits=tf.nn.relu(tf.layers.dense(logits,500))
    logits=tf.layers.dense(logits,vocab_len)
    sftm=tf.nn.softmax(logits)
    return logits,fw,bw,sftm


In [0]:
def concat(oba,ofa,obq,ofq):
  hidden_input_fw_=tf.concat((ofq,ofa),axis=-1)
  hidden_input_bw_=tf.concat((obq,oba),axis=-1)
# printop=tf.print("ques",hidden_input_bw_)
# with tf.control_dependencies([printop]):
  f=tf.nn.rnn_cell.LSTMStateTuple(hidden_input_fw_[0][0],hidden_input_fw_[0][1])
  b=tf.nn.rnn_cell.LSTMStateTuple(hidden_input_bw_[0][0],hidden_input_bw_[0][1])
  return f,b


In [37]:
max_seq_dist

43

In [38]:
tf.reset_default_graph()
teacher_forcing=0.50
keep_prob= tf.placeholder(tf.float32)
drpouts= tf.placeholder(tf.float32)
X_ques = tf.placeholder(tf.int32, shape=[None,max_seq_ques+1])
X_ans = tf.placeholder(tf.int32, shape=[None,max_seq_ans+1])
X_dist = tf.placeholder(tf.int32, shape=[None,max_seq_dist+1])
X_dist_d = tf.placeholder(tf.int32, shape=[None,1])

Y=tf.placeholder(tf.int32, shape=[None,max_seq_dist+1])
# lr=X_dist_d = tf.placeholder(tf.float32)
word_embeddings=tf.Variable(embedding_matrix,name="emb",trainable=False,dtype=tf.float32)
x_ques= tf.nn.embedding_lookup(word_embeddings,X_ques)
x_ans= tf.nn.embedding_lookup(word_embeddings,X_ans)

ofq,obq=ques_emb(x_ques,drpouts,keep_prob)
ofa,oba=ans_emb(x_ans,drpouts,keep_prob)
f,b=concat(oba,ofa,obq,ofq)


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
fw <class 'list'>


In [0]:
def encoder():  
  x_dist= tf.nn.embedding_lookup(word_embeddings,X_dist)
  for ind in range(max_seq_dist+1):
    if ind==0:
      logits,f1,b1,sftm=main_network(f,b,x_dist[:,:1,:],drpouts,keep_prob)
      loss_encoder=[logits]
      # nv=[]
      teq=tf.argmax(sftm,axis=-1)
      tq=tf.nn.embedding_lookup(word_embeddings,teq)
      # nv.append(teq)
    else:    
      pro=np.random.rand() 
      if pro<teacher_forcing:
        logits,f1,b1,sftm=main_network(f1[0],b1[0],tq,drpouts,keep_prob)
        loss_encoder.append(logits)  
      else:   
        print(ind,end=" ")
        logits,f1,b1,sftm=main_network(f1[0],b1[0],x_dist[:,ind:ind+1,:],drpouts,keep_prob)
        loss_encoder.append(logits)   
      teq=tf.argmax(sftm,axis=-1)
      tq=tf.nn.embedding_lookup(word_embeddings,teq)  
      # nv.append(teq)

  loss_encoder=tf.concat(loss_encoder,1) 
  return loss_encoder



In [0]:
def decoder():  
  x_dist= tf.nn.embedding_lookup(word_embeddings,X_dist_d)
  for ind in range(max_seq_dist+1):
    if ind==0:
      logits,f1,b1,sftm=main_network(f,b,x_dist[:,:1,:],drpouts,keep_prob)
      loss_val=[logits]
      nv=[]
      teq=tf.argmax(sftm,axis=-1)
      tq=tf.nn.embedding_lookup(word_embeddings,teq)
      nv.append(teq)
    else:    
      logits,f1,b1,sftm=main_network(f1[0],b1[0],tq,drpouts,keep_prob)
      loss_val=[logits]
      teq=tf.argmax(sftm,axis=-1)
      tq=tf.nn.embedding_lookup(word_embeddings,teq)  
      nv.append(teq) 
  return nv,loss_val

In [41]:
loss_encoder=encoder()
dec_sen,loss_val=decoder()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
1 2 8 10 15 19 20 21 22 23 24 27 29 30 31 32 36 38 43 

In [42]:
type(loss_encoder),type(dec_sen)

(tensorflow.python.framework.ops.Tensor, list)

In [0]:
one_hot=tf.one_hot(indices=Y,depth=vocab_len,axis=-1)       
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot,logits=loss_encoder))
valiation_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot,logits=loss_val))
with tf.control_dependencies([loss_encoder]):
  optimizer = tf.train.AdamOptimizer(learning_rate =0.01).minimize(loss)  

In [44]:
data_feed_ans.shape,data_feed_ques.shape,data_feed_dist.shape,data_feed_label.shape

((25199, 15), (25199, 17), (25199, 44), (25199, 44))

In [52]:
saver=tf.train.Saver()
sess=tf.InteractiveSession()

EPOCH=300
BATCH_SIZE=128
# try:
saver.restore(sess,"/content/drive/My Drive/saved_model_value_labs/model1275.ckpt")
print("model restored")
# except:
#   sess.run(tf.global_variables_initializer()) 
#   print("not restored")  
for ep in tqdm(range(EPOCH)):
  for j in range(len(data_feed_ques)//BATCH_SIZE):
    sm=[]
    ques=data_feed_ques[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
    ans=data_feed_ans[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
    dist=data_feed_dist[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
    lab=data_feed_label[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
    train_loss,_= sess.run([loss,optimizer],feed_dict={X_ques:ques,X_ans:ans,X_dist:dist,Y:lab,keep_prob:1.0,drpouts:1.0})
    sm.append(train_loss)
    # print(train_loss,end=" ")
  print(ep," :",sum(sm)/len(sm))
  saver.save(sess,"/content/drive/My Drive/saved_model_value_labs/model"+str(ep+1276)+".ckpt")
  print("saved")

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/saved_model_value_labs/model1275.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/saved_model_value_labs/model1275.ckpt
  0%|          | 0/300 [00:00<?, ?it/s]

model restored
0  : 2.3916709423065186


  0%|          | 1/300 [03:09<15:44:11, 189.47s/it]

saved
1  : 2.386119842529297


...

 18%|█▊        | 55/300 [2:38:25<11:47:12, 173.19s/it]

saved
55  : 2.2848222255706787


 19%|█▊        | 56/300 [2:41:20<11:46:11, 173.65s/it]

saved


KeyboardInterrupt: ignored

In [0]:
# saver=tf.train.Saver()
# sess.run(tf.global_variables_initializer())
# sess=tf.InteractiveSession()
# saver.restore(sess,"/content/drive/My Drive/saved_model_value_labs/model.ckpt")
# print("restored")
# teacher_forcing=0.7
j=0
BATCH_SIZE=6300
# teacher_forcing=np.inf
ques=data_feed_ques_test[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
ans=data_feed_ans_test[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
# dist=data_feed_dist_test[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
dist=np.zeros((BATCH_SIZE,1),dtype=np.int32)
# lab=label_f_test[j*BATCH_SIZE:(j*BATCH_SIZE+BATCH_SIZE)]
dist[:,:1]=word2idx["sos"]
_= sess.run(dec_sen,feed_dict={X_ques:ques,X_ans:ans,X_dist_d:dist,keep_prob:1.0,drpouts:1.0})

In [0]:
word2idx["*"]=0
idx2w={}
for i,j in word2idx.items():
  idx2w[j]=i

In [55]:
sen=np.stack(_,1)
for j,i in enumerate(sen):
  print(j,end="   ")
  for k in i:
    print(idx2w[k[0]],end=" ")
  print()  

0   how of the , * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
1   the s the the the * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
2   coaches should learn students students students students students students students students students students students students students students transparency teaching teaching urgently teaching urgently teaching urgently urgently teaching urgently activities smokers teaching teaching teaching smokers * dual eos , students students students , * * 
3   it is the the the the eos , the the the the * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
4   dancing eos * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
5   the the the the the the eos eos the the the the the the * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
6   the the eos eos * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
7   they a

KeyboardInterrupt: ignored

In [51]:
 # sen=np.stack(lg,1)
for j,i in enumerate(data_feed_dist_test):
  print(j,end="   ")
  for k in i:
    print(idx2w[k],end=" ")
  print()  

0   sos the robin s songs , the birds habitat , how to protect birds * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
1   sos it s interesting to fight with the birds , you ca nt defeat the bird in the end , you should live with birds peacefully * * * * * * * * * * * * * * * * * * 
2   sos teachers are really important for students , the project works like a conveyor belt of information * * * * * * * * * * * * * * * * * * * * * * * * * * * 
3   sos it is a big city in space * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
4   sos jenna and i * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
5   sos fatigue toxinscould hardly be found in a labour s blood , albert eistein did nt feel worn out after a day s work , a mental worker s blood was filled with fatigue toxins * * * * * * * * * 
6   sos no beam is shown on the white paper , no light can be noticed inside the glass * * * * * * * * * * * * * * * * * * * * * * * * * *